# Notebook to Illustrate Grid Mismatch Between UA SWE Dataset and MRF Dataset

# Notebook Setup 

In [1]:
# import libraries
import geopandas as gpd
import pandas as pd
import folium
import s3fs
import xarray as xr
from geopy.distance import distance
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import numpy as np
from rasterio.transform import from_bounds

In [2]:
# define some functions

# function to calculate the transform of a ds that is missing it 
# needed for the wrf data 
def calc_transform(ds):
    transform = from_bounds(west=ds.lon.min().item(),
        south=ds.lat.min().item(),
        east=ds.lon.max().item(),
        north=ds.lat.max().item(),
        width=ds.dims["lon"],
        height=ds.dims["lat"],
        )
    return transform




# Create a map of Mount Baker and ~ 16 km radius around it 



In [3]:
# create a map of Mount Baker and ~5 mile radius around it
mount_baker_coords = (48.773, -121.8132)
radius_km = 16

# Calculate bounding box coordinates
north = distance(kilometers=radius_km).destination(mount_baker_coords, bearing=0)
south = distance(kilometers=radius_km).destination(mount_baker_coords, bearing=180)
east = distance(kilometers=radius_km).destination(mount_baker_coords, bearing=90)
west = distance(kilometers=radius_km).destination(mount_baker_coords, bearing=270)

# Initialize the map centered on Mount Baker
m = folium.Map(location=mount_baker_coords, zoom_start= 10, width='100', height='90%')

# Create bounding box as a variable
b_box = [
    (north.latitude, west.longitude),
    (north.latitude, east.longitude),
    (south.latitude, east.longitude),
    (south.latitude, west.longitude),
    (north.latitude, west.longitude)
]

# Add bounding box
folium.PolyLine(b_box, color="gray", weight=2.5, opacity=2).add_to(m)

# Display the map
m

# Add to the map the 4km by 4km grids used by UA data (Red Lines)

In [4]:
# import SWE data for one day 
var = "swe"
day_filter = "10-01-1996"
b_bronze = f"{var}-bronze"
zarr_store_url = f's3://{b_bronze}/{var}_all.zarr'
ds_swe = xr.open_zarr(store=zarr_store_url, consolidated=True).sel(time = day_filter)
#ds_swe    

In [5]:
# Clip the swe data to the above bounding box 

b_box_polygon = Polygon([(point[1], point[0]) for point in b_box])  
ds_swe.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace = True)
ds_swe.rio.write_crs("EPSG:4326", inplace=True)
ds_swe_clipped = ds_swe.rio.clip([b_box_polygon], ds_swe.rio.crs)

#ds_swe_clipped

In [6]:
# Extract the unique latitudes and longitudes and add them to the folium map 
latitudes = ds_swe_clipped['lat'].values
longitudes = ds_swe_clipped['lon'].values

for lat in latitudes:
    folium.PolyLine([(lat, longitudes.min()), (lat, longitudes.max())], color='red', weight=2).add_to(m)

for lon in longitudes:
    folium.PolyLine([(latitudes.min(), lon), (latitudes.max(), lon)], color='red', weight=2).add_to(m)

m


# Now Add the grid used by the WRF Data (blue lines)

In [7]:
var = "pr"
day_filter = "10-01-1996"
b_bronze = "wrf-bronze"
zarr_store_url = f's3://{b_bronze}/{var}_all.zarr'
ds_pr = xr.open_zarr(store=zarr_store_url, consolidated=True).sel(day = day_filter)
ds_pr = ds_pr.sortby("lat") 
transform = calc_transform(ds_pr)
ds_pr = ds_pr.rio.write_transform(transform, inplace=True)
ds_pr.rio.write_crs("EPSG:4326", inplace=True)
ds_pr_clipped = ds_pr.rio.clip([b_box_polygon], ds_pr.rio.crs) 
#ds_pr_clipped

/tmp/ipykernel_18785/254748258.py:10: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  width=ds.dims["lon"],
/tmp/ipykernel_18785/254748258.py:11: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  height=ds.dims["lat"],


In [8]:
# Extract the unique latitudes and longitudes and add them to the folium map 
latitudes = ds_pr_clipped['lat'].values
longitudes = ds_pr_clipped['lon'].values

for lat in latitudes:
    folium.PolyLine([(lat, longitudes.min()), (lat, longitudes.max())], color='blue', weight=2).add_to(m)

for lon in longitudes:
    folium.PolyLine([(latitudes.min(), lon), (latitudes.max(), lon)], color='blue', weight=2).add_to(m)

m